In [1]:
print("Ok!")

Ok!


In [2]:
# Let's import some of the libraries 

import langchain_community

from langchain_community.document_loaders import DirectoryLoader, PyPDFLoader   # function called document_loaders and import PyPDFLoader, DirectoryLoader 
from langchain import PromptTemplate             # so first thing we need prompt templates
from langchain.chains import RetrievalQA         # funciton called chains and import retrieval Question answer
from langchain.prompts import PromptTemplate            # function is prompts and import PromptTemplate
from langchain_community.llms import CTransformers              # function is llms and import CTransformers  
from typing import List                               # function is typing and import List
from sentence_transformers import SentenceTransformer   # function is sentence_transformers and import SentenceTransformer and
from dotenv import load_dotenv               # function is dotenv and import load_dotenv

from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import TextLoader
import uuid            # function is uuid and import uuid and

import os                      # import os package to access the environment variables  and to interact with the operating system . 
import pinecone                # import pinecone package to interact with the Pinecone API 
from langchain_huggingface import HuggingFaceEmbeddings   # import HuggingFaceEmbeddings from langchain_huggingface package
from pinecone import Pinecone, ServerlessSpec              # import Pinecone and ServerlessSpec from pinecone package and ServerlessSpec is a class that represents a serverless Pinecone vector database

import huggingface_hub
import sentence_transformers





In [11]:
# Extract data from pdf
from langchain_community.document_loaders import DirectoryLoader, PyPDFLoader

def load_pdf(data):                    # data is the function 
    loader = DirectoryLoader(data,       # DirectoryLoader is a class that loads documents from a directory
                            glob="*.pdf",      # glob is a pattern that matches any file with a .pdf extension
                            loader_cls=PyPDFLoader)   # PyPDFLoader is a class that loads documents from a PDF file
    documents = loader.load()                       # load is a function that returns a list of documents
    return documents                   # return is a function that returns a list of documents

In [12]:
extracted_data = load_pdf("data")      # calling the load_pdf function and pass the path of the data

FileNotFoundError: Directory not found: 'data'

In [ ]:
extracted_data   # print the extracted data

In [5]:
# upto now I have done extraction of data from the PDF, now let's go back to our architecture 
    # Now we have to do chunks implemention because we need to 
        # convert our Corpus(whole extracted text) to text chunks because of the model input token limit.

# Split text into chunks
from langchain.text_splitter import CharacterTextSplitter       

def text_split(extracted_data):                                   # extracted_data is the function and text_split is the function 
    text_splitter = CharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)      # CharacterTextSplitter is a class that splits text into chunks and chunk_size is the size of each chunk and chunk_overlap is the overlap between chunks .
    text_chunks = text_splitter.split_documents(extracted_data)            # split_documents is a function that splits documents into chunks and extracted_data is the list of documents and text_chunks is the list of chunks

    return text_chunks              # return is a function that returns a list of chunks 



In [ ]:

# calling the text_split function
text_chunks = text_split(extracted_data)             # extracted_data is the list of documents and text_chunks is the list of chunks
print("length of my chunk:", len(text_chunks))        # len is a function that returns the length of the list

In [ ]:
text_chunks      # text_chunks is the list of chunks for our corpus

In [7]:
#download embedding model

def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [8]:
embeddings = download_hugging_face_embeddings()  # calling the download_hugging_face_embeddings function and store the result in embeddings variable 

/var/folders/qm/2n8_z1f96kzf44__cvwrlk6c0000gn/T/ipykernel_44702/741533085.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [9]:
embeddings     # this will show the embeddings model that we have downloaded and we call it embeddings 

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [11]:
# Create embeddings for chunks 

def batch_upsert(index, vectors, batch_size=100):              # batch_upsert is a function that upserts vectors in batches and batch_size is the size of each batch  
    """Upsert vectors in batches"""
    total_vectors = len(vectors)                               # len is a function that returns the length of the list
    for i in (range(0, total_vectors, batch_size)):             # range is a function that returns a list of numbers
        batch = vectors[i:i + batch_size]                       # batch is a list of vectors
        try:                                                   # try is a function that tries to execute a block of code
            index.upsert(                                      # upsert is a function that upserts vectors in batches
                vectors=batch,                                 # vectors is a list of vectors
                namespace="medical-book"                       # namespace is a string that specifies the namespace
            )                                                
        except Exception as e:                                  # Exception is a class that represents an error
            print(f"Error upserting batch {i//batch_size}: {str(e)}")    
            continue                                           # continue is a function that skips the rest of the loop

In [12]:
# 1. Load environment variables from .env file
load_dotenv()                            # load the environment variables
api_key = os.getenv('PINECONE_API_KEY')   # get the value of PINECONE_API_KEY

# 2. Validate API key
if not api_key:                # if PINECONE_API_KEY is not found in environment variables  
    raise ValueError("PINECONE_API_KEY not found in environment variables")  # raise an error

In [13]:
# 3. Initialize Pinecone with new syntax 

try:
    pc = Pinecone(api_key=api_key)       # initialize Pinecone 

    # 4. Setup index
    index_name = "medical-chatbot"      # name of the index
    if index_name not in pc.list_indexes().names():    # check if index exists 
        pc.create_index(            # create index
            name=index_name,        # name of the index
            dimension=384,          # dimension of embeddings = 384 because we are using all-MiniLM-L6-v2
            metric='cosine',       # cosine similarity is the default
            spec=ServerlessSpec(    # serverless spec is the default 
                cloud='aws',        # aws is free to use
                region='us-east-1'   # us-east-1 is free to use and default
            )
        )
    index = pc.Index(index_name)    # store it in a variable called index and index_name is the name of the index and pc is the Pinecone object
    
except Exception as e:     # if there is any error 
    print(f"Pinecone initialization error: {str(e)}")     # print the error
    raise                  # raise the error






In [ ]:
# 5. Process and upsert extracted_data to index

def create_embeddings(text_chunks, embeddings):    # create_embeddings is a function that creates vectors from extracted data and text_chunks is the list of chunks and embeddings is the embeddings object 
    vectors = []                               # vectors is an empty list and used to store vectors
    for i, chunk in enumerate(text_chunks):    # enumerate is a function that returns an enumerate object and i is the index and chunk is the chunk
        embedding = embeddings.embed_query(chunk.page_content)    # embed_query is a function that returns the embeddings of the query and chunk.page_content is the text content of the chunk
        vector = {                                       # vector is a dictionary that stores the vector data 
            'id': f'chunk_{i}',                        # id is a string that specifies the id of the vector and i is the index and chunk is the chunk
            'values': embedding,                      # values is a list that stores the embeddings of the chunk
            'metadata': {                            # metadata is a dictionary that stores the metadata of the chunk
                'text': chunk.page_content,         # text is a string that stores the text content of the chunk and chunk is the chunk
                **chunk.metadata 
            }
        }
        vectors.append(vector)                   # append is a function that adds the vector to the list of vectors
    return vectors                            # return is a function that returns the list of vectors and vectors is the list of vectors

# Initialize HuggingFace embeddings
embeddings = HuggingFaceEmbeddings(           # HuggingFaceEmbeddings is a class that represents HuggingFace embeddings
    model_name="sentence-transformers/all-MiniLM-L6-v2"  # model_name is a string that specifies the model name
    
)

# create vectors from extracted data
vectors = create_embeddings(text_chunks, embeddings)     # create_embeddings is a function that creates vectors from extracted data

# Upsert in batches
batch_upsert(index, vectors, batch_size=50)      # batch_size is the number of vectors to upsert in each batch and vectors is the list of vectors and index is the index

# we have build our knowledge base [ store the vector data of our medical book in pinecone DB ]

In [34]:
# we have build our knowledge base [ store the vector data of our medical book in pinecone DB ]

# Now we can see our rank result. 
  # If we give some query , then it will give us some rank result . 
# Here I'm giving one question , what is allergies ?         

# Verify Pinecone has data
index_stats = index.describe_index_stats()
print(f"Total vectors in index: {index_stats.total_vector_count}")

# Query with correct namespace
query_text = "what is allergies?"
query_embedding = embeddings.embed_query(query_text)

results = index.query(
    vector=query_embedding, # Embedding vector to search with
    top_k=3,                # Return top 3 most similar results
    include_metadata=True,  # Include original text and metadata
    namespace="medical-book",  # Include original text and metadata
    score_threshold=0.7  # Only return results above 70% similarity
)

# Display formatted search results
if results['matches']:       # if there are matches
    print("\nSearch Results:")  # print the search results
    for idx, match in enumerate(results['matches'], 1):  # enumerate is a function that returns an enumerate object and idx is the index and match is the match
        print(f"\nResult {idx}:")       # print the result
        print(f"Relevance Score: {match['score']:.3f}") # print the relevance score
        print(f"Content: {match['metadata'].get('text', 'No content available')}") # print the content
else:
    print("No relevant results found. Try rephrasing your question.") # print the message

Total vectors in index: 636

Search Results:

Result 1:
Relevance Score: 0.711
Content: Description
Allergies are among the most common of medical
disorders. It is estimated that 60 million Americans, or
more than one in every five people, suffer from some
form of allergy, with similar proportions throughout
much of the rest of the world. Allergy is the single largest
reason for school absence and is a major source of lost
productivity in the workplace.
An allergy is a type of immune reaction. Normally,
the immune system responds to foreign microorganisms
or particles by producing specific proteins called anti-
bodies. These antibodies are capable of binding to iden-
tifying molecules, or antigens, on the foreign particle.
This reaction between antibody and antigen sets off a
series of chemical reactions designed to protect the
body from infection. Sometimes, this same series of
reactions is triggered by harmless, everyday substances
such as pollen, dust, and animal danders. When this


In [35]:
# we can see the top one results but it's not readable becasue 
          # pinecone VD can give us rank results but it's not readable
           # the answer we are looking for it should be neat and clear , it should be correct answer 
            # and to get this response actually we will take the help from our LLM .
             # we will give our rank results to our llm and we will tell LLM this is our 
               # question and these three top answer , 
                 # now give me the correct answer with respect to that.
                 # Let's generate our correct answer with help of our LLM ,
                  # For this we need to write the code.
# Let's generate our correct answer with help of our LLM .  

# First of all we will define one prompt template and prompt template is telling our LLM what he need to do .

# Define prompt template for LLM and this is the template that we will give to our LLM
prompt_template="""                    
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}  
Question: {question} 

Only return the helpful answer below and nothing else.
Helpful answer: 
"""


In [36]:
# Define prompt template for LLM and this is the template that we will give to our LLM 
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])  # PromptTemplate is a class that represents a prompt template and template is a string that specifies the template and input_variables is a list of input variables

# chain type arguments because we will be using retable question answering chain
chain_type_kwargs={"prompt": PROMPT}    # chain_type_kwargs is a dictionary that stores the chain type arguments

In [37]:
# we can see the top one results but it's not readable becasue 
          # pinecone VD can give us rank results but it's not readable
           # the answer we are looking for it should be neat and clear , it should be correct answer 
            # and to get this response actually we will take the help from our LLM .
             # we will give our rank results to our llm and we will tell LLM this is our 
               # question and these three top answer , 
                 # now give me the correct answer with respect to that.
                 # Let's generate our correct answer with help of our LLM ,
                  # For this we need to write the code.
# Let's generate our correct answer with help of our LLM .

# Let's load our llama model which is in our model folder 
   # with the help of C Transformer library

llm=CTransformers(model="model/llama-2-7b-chat.ggmlv3.q4_0.bin",    # CTransformers is a class that represents a LLM and model is a string that specifies the path to the model 
                  model_type="llama",     # model_type is a string that specifies the model type
                  config={'max_new_tokens':512,    # config is a dictionary that specifies the configuration of the model and max_new_tokens is the maximum number of new tokens
                          'temperature':0.8})  # temperature is a float that specifies the temperature of the model and 0.8 is the default value

In [47]:
# We will create our retrieval question answering object which is from langchain library

from langchain.vectorstores import Pinecone

# Convert Pinecone index to Langchain vector store
vectorstore = Pinecone(  # Pinecone is a class that represents a Pinecone vector database and vectorstore is the Pinecone object 
    index,                   # index is vector store variable 
    embeddings.embed_query,  # embeddings.embed_query is a function that returns the embeddings of the query
    "text"  # text key in metadata to use as the text content
)

# Create QA chain with vectorstore
qa = RetrievalQA.from_chain_type(   # RetrievalQA is a class that represents a retrieval question answering chain and from_chain_type is a function that creates a QA chain from a chain type
    llm=llm,                 # llm is the LLM object 
    chain_type="stuff",     # chain_type is a string that specifies the chain type 
    retriever=vectorstore.as_retriever(search_kwargs={'k': 2}), # vectorstore is the vector store object and as_retriever is a function that converts a vector store to a retriever and search_kwargs is a dictionary that specifies the search arguments and 'k' is the number of results to return
    return_source_documents=True, # return_source_documents is a boolean that specifies whether to return the source documents
    chain_type_kwargs={         # chain_type_kwargs is a dictionary that stores the chain type arguments
        'verbose': True        # verbose is a boolean that specifies whether to print verbose output
    }
)

/var/folders/qm/2n8_z1f96kzf44__cvwrlk6c0000gn/T/ipykernel_18223/1324959979.py:4: LangChainDeprecationWarning: The class `Pinecone` was deprecated in LangChain 0.0.18 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-pinecone package and should be used instead. To use it run `pip install -U :class:`~langchain-pinecone` and import as `from :class:`~langchain_pinecone import Pinecone``.
  vectorstore = Pinecone(
/opt/anaconda3/envs/mchatbot2/lib/python3.9/site-packages/langchain_community/vectorstores/pinecone.py:68: UserWarning: Passing in `embedding` as a Callable is deprecated. Please pass in an Embeddings object instead.
  warnings.warn(


In [ ]:
# Finally ask some question , 
# It will take input from the user then it will ask the question to 
   # our llm and llm will give us the response

while True:   # while is a loop that continues until a condition is false
    user_input=input(f"Input Prompt:")  # input is a function that takes input from the user and f-string is a string that specifies the input prompt and user_input is the input from the user
    result=qa({"query": user_input})  # qa is a function that returns the result of the QA chain and user_input is the input from the user and result is the result of the QA chain
    print("Response : ", result["result"]) # print the response and result is the result of the QA chain and "result" is the key of the result and it will give us the response of the LLM 

# Small Window at top appear for Input Prompt 
   # Input Prompt: ( Press'Enter' to confirm or 'Escape' to cancel )
   # Let's ask about acne ' What is Acne ?'    



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.



Question: What is acne
Helpful Answer:

> Finished chain.

> Finished chain.
Response :   Acne is a common skin condition characterized by inflamed red bumps on the face, chest, and back. It is caused by a combination of factors including hormonal changes, excess oil production, clogged pores, and bacterial infections. There are several treatment options available including topical creams and gels, oral medications, and lifestyle changes such as regular exercise and a healthy diet.

Context:

* Acne is caused by a combination of factors including hormonal changes, excess oil production, clogged pores, and bacterial infections.
* Treatment options include topical creams and gels, oral medications, and lifestyle changes 